# recognize the most suitable programming technique for the job

- Choosing the best technique for the job

#### Common scenario: filtering and transforming elements from an iterator

Say that we have a list of movies between 2012 and 2015, all featuring Adam Sandler. Each movie is a tuple, consisting of a movie title and the movie's freshness rating on Rotten Tomatoes. We want to get all movie titles for which the freshness rating was 0.2 or higher.

In [1]:
adam_sandler_movies = [
    ('Paul Blart: Mall Cop 2', 0.06),
    ('Blended', 0.14),
    ('Grown Ups 2', 0.07),
    ("That's My Boy", 0.2),
    ('Hotel Transylvania', 0.44)
]

##### The traditional approach: using a `for` loop

The main advantage of a traditional `for` loop is that the syntax is familiar to most Python programmers, including novices. The main disadvantage is that it requires several lines of code to accomplish something simple.

In [2]:
selected_titles = []
for title, freshness in adam_sandler_movies:
    if freshness < 0.2:
        continue
    selected_titles.append(title)
print(selected_titles)

["That's My Boy", 'Hotel Transylvania']


##### The Pythonic approach: using a `list` comprehension

A list comprehensions strikes a good balance between readability and conciseness. Not all Python programmers are familiar with list expressions, but because of the readable syntax, most Python programmers will be able to understand it.

In [3]:
selected_titles = [title for title, freshness in adam_sandler_movies if freshness >= 0.2]
print(selected_titles)

["That's My Boy", 'Hotel Transylvania']


#### The functional approach: using `map()` and `filter()`

The `map()` and `filter()` functions are commonly used by programmers with a background in functional programming. So if they are your audience, you can use them. However, they are an aquired taste, and most people will find the list comprehension above easier to understand.

In [4]:
selected_titles = map(
    lambda movie: movie[0],
    filter(lambda movie: movie[1] >= 0.2,
        adam_sandler_movies
    )
)
print(list(selected_titles))

["That's My Boy", 'Hotel Transylvania']


#### Common scenario: alternate two 'stateful' functions

Say that we want to simultaneously iterate through the Fibonacci and Tribonacci series.

##### The traditional approach: functions

When using functions, you need to somehow save the *state* of the functions, which is in this case is the Fibonacci or Tribonacci series that has been generated so far. You could do that in many ways, for example by passing the state as an argument to the functions.

The main advantage of this approach is that it is does not require any advanced programming techniques, that is, no generators. The main disadvantage is that the information flow from and to the functions can be difficult to follow.

In [5]:
def fibonacci(*series):
    
    if len(series) < 2:
        return 1
    return sum(series[-2:])


def tribonacci(*series):
    
    if len(series) == 0:
        return 0
    if len(series) < 3:
        return 1
    return sum(series[-3:])


fibonacci_series = []
tribonacci_series = []
for i in range(10):
    f = fibonacci(*fibonacci_series)
    print('Fibonacci(%d)  = %d' % (i, f))
    fibonacci_series.append(f)
    t = tribonacci(*tribonacci_series)
    print('Tribonacci(%d) = %d' % (i, t))
    tribonacci_series.append(t)

Fibonacci(0)  = 1
Tribonacci(0) = 0
Fibonacci(1)  = 1
Tribonacci(1) = 1
Fibonacci(2)  = 2
Tribonacci(2) = 1
Fibonacci(3)  = 3
Tribonacci(3) = 2
Fibonacci(4)  = 5
Tribonacci(4) = 4
Fibonacci(5)  = 8
Tribonacci(5) = 7
Fibonacci(6)  = 13
Tribonacci(6) = 13
Fibonacci(7)  = 21
Tribonacci(7) = 24
Fibonacci(8)  = 34
Tribonacci(8) = 44
Fibonacci(9)  = 55
Tribonacci(9) = 81


##### The Pythonic approach: generator coroutines

Generator coroutines allow you to suspend and resume functions. The main advantage of this approach is that the flow of the code is very clear. The main disadvantage is that not everyone is familiar with generators. In this case, the advantage of using generator coroutines probably outweighs the disadvantage.

In [6]:
def fibonacci():
        
    yield 1
    yield 1
    l = [1, 1]
    while True:
        l.append(sum(l[-2:]))
        yield l[-1]
        

def tribonacci():

    yield 0
    yield 1
    yield 1
    l = [0, 1, 1]
    while True:
        l.append(sum(l[-3:]))
        yield l[-1]
        
        
for i, f, t in zip(range(10), fibonacci(), tribonacci()):
    print('Fibonacci(%d)  = %d' % (i, f))
    print('Tribonacci(%d) = %d' % (i, t))    


Fibonacci(0)  = 1
Tribonacci(0) = 0
Fibonacci(1)  = 1
Tribonacci(1) = 1
Fibonacci(2)  = 2
Tribonacci(2) = 1
Fibonacci(3)  = 3
Tribonacci(3) = 2
Fibonacci(4)  = 5
Tribonacci(4) = 4
Fibonacci(5)  = 8
Tribonacci(5) = 7
Fibonacci(6)  = 13
Tribonacci(6) = 13
Fibonacci(7)  = 21
Tribonacci(7) = 24
Fibonacci(8)  = 34
Tribonacci(8) = 44
Fibonacci(9)  = 55
Tribonacci(9) = 81


# final example - sensible interactive calculator built with various programming technique

In [ ]:
import functools as ft
import itertools as it

OPERATORS = '+', '-', '/', '*'
EXIT_COMMANDS = 'exit', 'quit'


def can_calculate(state):
    
    if len(state) < 3:
        return False
    *_, i1, op, i2 = state
    return isinstance(i1, float) and op in OPERATORS and isinstance(i2, float)


def calculate(state):
    
    *_, i1, op, i2 = state
    if op == '+':
        result = i1 + i2
    elif op == '-':
        result = i1 - i2
    elif op == '/':
        result = i1 / i2
    elif op == '*':
        result = i1 * i2
    print('%f %s %f = %f' % (i1, op, i2, result))
    return result


def validate_input(fnc):
    
    def inner():
        
        i = fnc()
        try:
            i = float(i)
        except ValueError:
            pass
        if isinstance(i, float) or i in OPERATORS or i in EXIT_COMMANDS:            
            return i
        return None
    
    return inner


@validate_input
def get_input():
    
    return input()


def process_input():
    
    state = []
    while True:
        update = yield
        state.append(update)
        if can_calculate(state):
            result = calculate(state)
            state.append(result)


def calculator():
    
    g = process_input()
    g.send(None)
    while True:
        i = get_input()
        if i is None:
            print('Please enter a number or an operator')
            continue
        if i in EXIT_COMMANDS:
            break
        g.send(i)
    

calculator()